## [Day 14](https://adventofcode.com/2020/day/14)

This problem has some method of storing binary numbers in which we force particular digits to remain as certain values. My understanding is that the mask. We also have different addresses for which the the data is supposed to be stored.

In [1]:
import pandas as pd
import numpy as np
import re
instr = open('../inputs/d14.txt').read().splitlines()
instr[:5]

['mask = X111000X0101100001000000100011X0000X',
 'mem[4812] = 133322396',
 'mem[39136] = 1924962',
 'mem[35697] = 29912136',
 'mem[41065] = 2558851']

So maybe I'll go back to using pandas to get this done although it seems more complicated (as usual)

In [2]:
masks = ['' if x.find('mask') == -1 else x[7:] for x in instr]

masks[:5]

['X111000X0101100001000000100011X0000X', '', '', '', '']

In [3]:
addresses, values = [], []

def process_instr(x):
    m = re.match(r"mem\[(?P<address>\d+)\] = (?P<value>\d+)", x)
    if m:
        addresses.append(m.group('address'))
        values.append(m.group('value'))
    else:
        addresses.append('')
        values.append('')

for i in range(len(instr)):
    process_instr(instr[i])
addresses[:5]

['', '4812', '39136', '35697', '41065']

In [4]:
values[:5]

['', '133322396', '1924962', '29912136', '2558851']

In [5]:
# Now we can make a data frame out of this:
instr_df = pd.DataFrame({'address' : addresses, 'value' : values, 'bit_mask' : masks})
instr_df.bit_mask = instr_df.bit_mask.replace(to_replace = '', value = None).ffill()
instr_df = instr_df.query("address != ''").reset_index(drop = True)
instr_df.head(7)

,address,value,bit_mask
0,4812,133322396,X111000X0101100001000000100011X0000X
1,39136,1924962,X111000X0101100001000000100011X0000X
2,35697,29912136,X111000X0101100001000000100011X0000X
3,41065,2558851,X111000X0101100001000000100011X0000X
4,38134,481,11001101X110000X010X01101100X1X0X001
5,53084,5470,11001101X110000X010X01101100X1X0X001
6,37619,2696,11001101X110000X010X01101100X1X0X001


Alright so that took quite some time but learned a few things. Next I think I wanna just turn all those values into padded binary strings.

In [6]:
def to_bin(x):
    x = int(x)
    output = ''
    for i in range(36):
        if x >= 2**(35-i):
            output += '1'
            x -= 2**(35-i)
        else:
            output += '0'
    return output

instr_df['value_bin'] = instr_df['value'].copy().map(to_bin)
instr_df.head(7)

,address,value,bit_mask,value_bin
0,4812,133322396,X111000X0101100001000000100011X0000X,000000000111111100100101011010011100
1,39136,1924962,X111000X0101100001000000100011X0000X,000000000000000111010101111101100010
2,35697,29912136,X111000X0101100001000000100011X0000X,000000000001110010000110110001001000
3,41065,2558851,X111000X0101100001000000100011X0000X,000000000000001001110000101110000011
4,38134,481,11001101X110000X010X01101100X1X0X001,000000000000000000000000000111100001
5,53084,5470,11001101X110000X010X01101100X1X0X001,000000000000000000000001010101011110
6,37619,2696,11001101X110000X010X01101100X1X0X001,000000000000000000000000101010001000


Next step will be to set up a dictionary for the addresses. We initialize it with zeroes I believe.

In [7]:
addresses_u = instr_df.address.unique().tolist()
zero_bin = to_bin('0')
addresses_i = [zero_bin for i in range(len(addresses_u))]
addresses_dict = dict(zip(addresses_u, addresses_i))
#addresses_dict['4812']

Alright, last setup is to make a function that takes in the binary value and the bit mask and then smooshes them.

In [8]:
def apply_mask(val, mask):
    # forgot that you cannot do positional assignment with
    # strings so we have to go to a list:
    val = [char for char in val]
    for i in range(len(mask)):
        if mask[i] != 'X':
            val[i] = mask[i]
    return ''.join(val)
apply_mask(to_bin('1'), instr_df.bit_mask.iloc[0])

'011100000101100001000000100011000001'

Next we iterate

In [9]:
# Note that index and row are given by position, not name here
for index, row in instr_df.iterrows():
    addresses_dict[row['address']] = apply_mask(row['value_bin'], row['bit_mask'])

# Then the solution comes from summing over the dictionary values in base 10
def to_10(x):
    tot = 0
    for i in range(len(x)):
        n = int(x[i])
        tot += n*2**(len(x)-i-1)
    return(tot)
to_10('1101')    

13

In [10]:
solution = 0
for val in addresses_dict.values():
    solution += to_10(val)
solution

8570568288597

Okay so that was somewhat laboreous but not that conceptually hard.... let's see what part 2 has in store

### Part 2

Ok this is more difficult. The twist on this is that instead of having the masks change the values, they modify the addresses which now should be see in binary.
In the mask, the symbols are seen as:  
* 1: forces that bit to a 1
* 0: leaves the bit alone
* X: makes the bit a wildcard meaning it can be either a zero or one
As a result of the wildcards, we will assign the new value to (frequently) more than one memory address.

So as I'm thinking about this, as a not trained CS person, I'm thinking there should be some way of grouping the addresses or forming some kinda network where we can remove addresses from consideration as we look through.


In [11]:
# first thing is I want a binary version of the addresses. and I want to reform the
# dictionary of stored values
addresses_dict_bin = {}

instr_df['address_bin'] = instr_df['address'].map(to_bin)
instr_df.head()

,address,value,bit_mask,value_bin,address_bin
0,4812,133322396,X111000X0101100001000000100011X0000X,000000000111111100100101011010011100,000000000000000000000001001011001100
1,39136,1924962,X111000X0101100001000000100011X0000X,000000000000000111010101111101100010,000000000000000000001001100011100000
2,35697,29912136,X111000X0101100001000000100011X0000X,000000000001110010000110110001001000,000000000000000000001000101101110001
3,41065,2558851,X111000X0101100001000000100011X0000X,000000000000001001110000101110000011,000000000000000000001010000001101001
4,38134,481,11001101X110000X010X01101100X1X0X001,000000000000000000000000000111100001,000000000000000000001001010011110110


Also make the new version of the mask application:

In [12]:
def apply_mask2(val, mask):
    val = [char for char in val]
    for i in range(len(mask)):
        if mask[i] in ['X', '1']:
            val[i] = mask[i]
    return ''.join(val)
apply_mask2(instr_df.address_bin.iloc[0], instr_df.bit_mask.iloc[0])

'X111000X0101100001000001101011X0110X'

My initial assumption from this problem was that the only plausible addresses were those in the list but after failing to produce a reasonable answer with that assumption, I'm ditching it. As a result, we're going to form the combinations that fit the masked address and return them with this function:

In [13]:
def get_combos(x):
    # Store the resultant strings in this:
    if x.find('X') > -1:
        combos = [x.replace('X', '0', 1), x.replace('X', '1', 1)]
        return get_combos(combos[0]) + get_combos(combos[1])
    else:
        return [x]
        
get_combos('1X00X')

['10000', '10001', '11000', '11001']

So now we gotta run through each row of the datafram and update the dictionary

In [14]:
for index, row in instr_df.iterrows():
    # apply the mask, make a list of addresses to assign to:
    fuzzy_address = apply_mask2(row['address_bin'], row['bit_mask'])    
    all_addresses = get_combos(fuzzy_address)
        
    # finally update those addresses values:
    for address in all_addresses:
        addresses_dict_bin.update({address : row['value_bin']})

In [15]:
solution2 = 0
for val in addresses_dict_bin.values():
    solution2 += to_10(val)
solution2

3289441921203

### EXPERIMENTS

In [16]:
# 2383538661343 too low

In [17]:
?enumerate

In [18]:
x = [4, 5, 6]
for i, j in enumerate(x):
    print(i)

0
1
2
